In [1]:
import pandas as pd


In [6]:
data = pd.read_parquet("datasets/final/dataset_v0.parquet")

In [7]:
data.head()

,code,key,tglang
2,\n# syntax: GAWK -f ASCENDING_PRIMES.AWK\nBEG...,11,TGLANG_LANGUAGE_AWK
3,\n// Ascending primes. Nigel Galloway: April ...,36,TGLANG_LANGUAGE_FSHARP
5,"package main\n \nimport (\n ""fmt""\n ""rcu...",39,TGLANG_LANGUAGE_GO
8,using Combinatorics\nusing Primes\n \nfunction...,51,TGLANG_LANGUAGE_JULIA
9,local function is_prime(n)\n if n < 2 then re...,57,TGLANG_LANGUAGE_LUA


In [8]:
used_keys = list(
    map(
        lambda x: x[0],
        filter(
            lambda x: x[1] >= 2,
            ((key, value) for key, value in df["key"].value_counts().items())
        )
    )
)

len(used_keys)

98

In [10]:
data = data[data["key"].isin(used_keys)]
data.shape

(67666, 3)

In [11]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import numpy as np

class RandomModel:
    def __init__(self):
        pass
    def fit(self, X, y):
        self.X = X
        self.y = y

        self.targets = np.fromiter(np.unique(y), dtype=int)

        return self
    def predict(self, X):
        return np.random.choice(self.targets, len(X))

In [14]:
X_train, X_test, y_train, y_test = train_test_split(data["code"], data["key"], test_size=0.10, random_state=137)

random_model = RandomModel()
random_model.fit(X_train, y_train)
y_predict = random_model.predict(X_test)

accuracy_score(y_test, y_predict)


0.010196542042263928

In [17]:
small_data = data.copy()
small_data["code"] = small_data["code"].map(lambda x: x[:4096])

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    min_df=5,
    max_features=100000
)

X_train, X_test, y_train, y_test = train_test_split(
    small_data["code"],
    small_data["key"], test_size=0.10, random_state=137)

tfidf = tfidf.fit(X_train)

X_train = tfidf.transform(X_train)
X_test  = tfidf.transform(X_test)

X_train.shape, X_test.shape

((60899, 43348), (6767, 43348))

In [19]:
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier

In [24]:
sgd = SGDClassifier(loss="log_loss")
svc = LinearSVC()

In [25]:
from sklearn.metrics import accuracy_score,hamming_loss,precision_score,recall_score,f1_score
def print_metrics_report(clf,y_test,y_pred):
    print("Clf: ", clf.__class__.__name__)
    print('Exact Match Ratio: {0}'.format(np.round(accuracy_score(y_test, y_pred, normalize=True, sample_weight=None)*100,2)))
    print('Accuracy: {0}'.format(accuracy_score(y_test, y_pred)))
    print('Hamming loss: {0}'.format(np.round(hamming_loss(y_test, y_pred)*100,2))) 
    print('Recall: {0}'.format(np.round(precision_score(y_test, y_pred, average='micro')*100,2)))
    print('Precision: {0}'.format(np.round(recall_score(y_test,y_pred, average='micro')*100,2)))
    print('F1 Measure: {0}'.format(np.round(f1_score(y_test, y_pred, average='micro')*100,2))) 
    print('-------------')
    print("")

In [26]:
for classifier in [sgd, svc]:
    clf = OneVsRestClassifier(classifier)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print_metrics_report(classifier,y_test,y_pred)

Clf:  SGDClassifier
Exact Match Ratio: 78.2
Accuracy: 0.7820304418501551
Hamming loss: 21.8
Recall: 78.2
Precision: 78.2
F1 Measure: 78.2
-------------



/Users/platon.fedorov/opt/anaconda3/envs/tgml/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/platon.fedorov/opt/anaconda3/envs/tgml/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/platon.fedorov/opt/anaconda3/envs/tgml/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/Users/platon.fedorov/opt/anaconda3/envs/tgml/lib/python3.10/site-packages/sklearn/svm/_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the v

Clf:  LinearSVC
Exact Match Ratio: 87.08
Accuracy: 0.8708438007979903
Hamming loss: 12.92
Recall: 87.08
Precision: 87.08
F1 Measure: 87.08
-------------



In [27]:
model_0 = clf.estimators_[0]

model_0.n_features_in_, model_0.coef_

(43348,
 array([[ 0.16673093, -0.01976526, -0.06637467, ...,  0.        ,
          0.        ,  0.        ]]))